# 线性回归

In [2]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

## 线性回归的从零开始实现

###  生成数据集

In [3]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.randn(num_examples, num_inputs, dtype=torch.float32)
labels = true_w[0] * features[:,0] + true_w[1]*features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.02, size=labels.size()), dtype=torch.float32)

In [4]:
print(features[0], labels[0])

tensor([0.0860, 0.8108]) tensor(1.5851)


### 读取数据

In [6]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    
    for i in range(0, num_examples, batch_size):
        j = torch.LongTensor(indices[i : min(i+batch_size, num_examples)])
        
        ## index_select(raw/col, indices)
        # 第一个参数0表示按行索引，1表示按列进行索引
        # 第二个参数是一个tensor， 表示索引号
        # yield关键字类似于return， 但是不同的是 当下一次调用这个函数的时候从yield返回
        yield features.index_select(0, j), labels.index_select(0, j)
                             

In [10]:
batch_size = 10
for X,y in data_iter(batch_size, features, labels):
    print(X, y)
    break

tensor([[ 1.1639,  1.4551],
        [-0.8685, -2.4034],
        [ 1.3185,  1.5825],
        [-0.8477,  0.2335],
        [ 1.0285, -0.7561],
        [ 0.4180,  0.2534],
        [ 1.0471,  0.9600],
        [-1.5523, -1.4421],
        [-0.7108, -1.7239],
        [-0.4439, -1.3587]]) tensor([ 1.5785, 10.6546,  1.4294,  1.7404,  8.8446,  4.1530,  3.0372,  5.9875,
         8.5957,  7.9186])


### 初始化模型参数

In [11]:
w = torch.tensor(np.random.normal(0, 0.01, size=(num_inputs, 1)), dtype = torch.float32)
b = torch.zeros(1, dtype = torch.float32)

In [12]:
w.requires_grad_(requires_grad = True)
b.requires_grad_(requires_grad = True)

tensor([0.], requires_grad=True)

### 定义模型

In [13]:
def linreg(X, w, b):
    # matrix multi 矩阵乘法
    return torch.mm(X, w) + b

### 定义损失函数

In [14]:
def squared_loss(y_hat, y):
    # 注意这里返回的是向量
    return (y_hat - y.view(y_hat.size())) ** 2 /2

### 定义优化器

In [15]:
def sgd(params, lr, batch_size):
    for param in params:
        param.data -=  lr * param.grad / batch_size

### 训练模型

In [27]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y).sum()
        l.backward()
        sgd([w, b], lr, batch_size)
        
        
        w.grad.data.zero_()
        b.grad.data.zero_()
        
    train_l = loss(net(features, w, b), labels)
    print("epoch %d, loss %f" % (epoch+1, train_l.mean()))

epoch 1, loss 0.000201
epoch 2, loss 0.000201
epoch 3, loss 0.000201


In [28]:
print(true_w, '\n', w)
print(true_b, '\n', b)

[2, -3.4] 
 tensor([[ 2.0010],
        [-3.3994]], requires_grad=True)
4.2 
 tensor([4.2010], requires_grad=True)
